In [2]:
import os
import pdfplumber
import glob
import re
import pandas as pd
import numpy as np

### Extract affordable housing financing data from Ohio PDFs


In [10]:
#filepath to PDFs
filepath = 'Ohio 2019 Applications/'
# create a list of filepaths 
files = glob.glob(filepath+"*.pdf", recursive = True)

In [13]:
test = "AmberlySquareApts.pdf"

In [14]:
pdf = pdfplumber.open(test)
#print(len(pdf.pages[0].extract_tables()[0]))
df = pd.DataFrame(pdf.pages[0].extract_tables()[0])

In [59]:
wage = df[38:][1][51] 


In [63]:
wage[17:]

'Ohio Prevailing Wage'

The pdf reader does not correctly extract tables from these documents, however it does pull data out in a relativley predictable pattern. Below I check for files that have the same structure - only 4 out of 121 do not, so instead of worry about the edge cases I store them in the list "broken files" and will deal with those manually.

In [109]:
for file in files:
    pdf = pdfplumber.open(file)
    #print(len(pdf.pages[0].extract_tables()[0]))
    df = pd.DataFrame(pdf.pages[0].extract_tables()[0])
    if df.shape[0] > 40 and df.iloc[37].values[1] == 'Construction Financing Sources':
        continue
    else:
        print(file, df.shape)

Ohio 2019 Applications/MayflowerManor.pdf (56, 14)
Ohio 2019 Applications/BerwynEastPlace.pdf (25, 9)
Ohio 2019 Applications/AppleseedSeniorApartments.pdf (35, 14)
Ohio 2019 Applications/SW-DetroitShorewayHomes.pdf (35, 13)


In [111]:
broken_files = ["Ohio 2019 Applications/MayflowerManor.pdf",
"Ohio 2019 Applications/BerwynEastPlace.pdf",
"Ohio 2019 Applications/AppleseedSeniorApartments.pdf",
"Ohio 2019 Applications/SW-DetroitShorewayHomes.pdf"]

files = list(set(files) - set(broken_files)) #remove broken files from master set of files 

In [25]:
def scrape_ohio(files):
    """
    Inputs:
        Files (list): List of full filepaths to the PDFs for Virginia. 
    This file notices that while the extract table function of the PDF reader does not exactly work
    in capturing the correct tables, the data follows a similar pattern and so it slices into 
    the files and grabs the relevent info from "Construction Financing Sources" 
    and "Permenant Financing Sources"
    """
    money = []
    wage_requirement = []
    for file in files:
        pdf = pdfplumber.open(file)
        df = pd.DataFrame(pdf.pages[0].extract_tables()[0])
        money_type = df.loc[37,1] #construction
        file_name = re.split("/", file)[-1] #regex to pull the filename from the filepath
        file_name = file_name[0:-4]

        for row in df[38:][1]:
            if 'TOTAL' in row:
                break
            else:
                print(row, "i am a row")
                money.append([x.strip() for x in row.split('$')]+[money_type]+[file_name])
                wage_r = df[38:][1][51][17:]
                wage_requirement.append(wage_r + [file_name])
        money_type = df.loc[37,6] #permanent
        for row in df[38:][6]:
            if 'TOTAL' in row:
                break
            else:
                money.append([x.strip() for x in row.split('$')]+[money_type]+[file_name])

    money_df = pd.DataFrame(money, columns=['Source',"Amount of Funds","Funding Type","File Name"])
    money_df = money_df[~money_df["Amount of Funds"].str.contains("-")] #drop the blanks 
    wage_requirement_df = []
     
    
    return money_df 

In [111]:
df = pd.read_csv("Ohio_scraped_financing_info.csv")

In [115]:
awards = awards.str.replace(' ', '')
#df.columns = df.columns.str.replace(' ', '')

In [118]:
awards = awards.iloc[1:]

In [112]:
df

,Unnamed: 0,Source,Amount of Funds,Funding Type,File Name
0,0,Tax Credit Equity,"1,765,000.85",Construction Financing Sources,CedarRedevelopment
1,1,HDAP,"1,500,000.00",Construction Financing Sources,CedarRedevelopment
2,2,Deferred Developer Fee,"669,512.00",Construction Financing Sources,CedarRedevelopment
3,3,Construction Loan,"10,046,087.15",Construction Financing Sources,CedarRedevelopment
4,4,Other1,"500,000.00",Construction Financing Sources,CedarRedevelopment
...,...,...,...,...,...
1295,1381,Tax Credit Equity,"6,707,586",Permanent Financing Sources,SW-DetroitShorewayHomes
1296,1382,Deferred Developer Fee,"58,445",Permanent Financing Sources,SW-DetroitShorewayHomes
1297,1383,"Permanent First Loan, Hard Debt","390,000",Permanent Financing Sources,SW-DetroitShorewayHomes
1298,1384,Other1,"600,000",Permanent Financing Sources,SW-DetroitShorewayHomes


In [136]:
#drop duplicates 
df1 = df[~df["File Name"].str.contains("(1)")]
df = pd.read_csv("Ohio_scraped_financing_info.csv")

In [144]:
#write to csv
df1.to_csv("Ohio_scraped_financing_info.csv")

In [ ]:
df = pd.read_csv("Ohio_scraped_financing_info.csv")
#read in list of those who actually won awards, this will also filter out the 4% and bond gap programs 
awards = pd.read_csv("2019-HTC-CompetitiveAwards.csv")
# link to data: https://ohiohome.org/ppd/documents/2019-HTC-CompetitiveAwards.xlsx
awards = awards["Unnamed: 1"]#drop this if col doesn't exist
awards = awards.dropna()
#now we have list of just those we want to keep 
awards = awards.str.replace(' ', '') #drop whitespace so place names match
awards = awards.iloc[1:]
df = df[df["Source"]!='GP Capital Contribution']
df = df[df["Source"]!='GP Equity'] #drop GP contributions 

#filter for only ones we want 
df = df[df['File Name'].isin(awards)]
df.to_csv("Ohio_scraped_financing_info_Awarded_Only.csv")

In [123]:
df.to_csv("Ohio_scraped_financing_info_Awarded_Only.csv")

In [3]:
import pandas as pd
df = pd.read_csv("Ohio_scraped_financing_info_Awarded_Only.csv")

In [6]:
df["Source"].value_counts()

Tax Credit Equity                    56
Deferred Developer Fee               44
Other1                               41
Construction Loan                    29
Other2                               29
Permanent First Loan, Hard Debt      24
Other3                               16
HDAP: OHTF/HOME                      11
HDAP                                 11
Other4                                8
Permanent Second Loan                 7
Other5                                5
OHFA HDL                              3
Historic Tax Credit Equity            3
Construction Period Income            2
Cuyahoga County Funds                 2
City of Cleveland Funds               2
Acquired Reserves                     2
Seller Note                           2
GP Equity                             2
City of Columbus/Franklin County      1
FHLB Cin. AHP                         1
DDF and Other                         1
Donations from Huntington and CHN     1
Stark County HOME                     1


In [11]:
df["Source"].unique()

array(['Tax Credit Equity', 'Construction Loan', 'Other1', 'Other2',
       'Other4', 'Deferred Developer Fee', 'HDAP', 'HDAP: OHTF/HOME',
       'Permanent First Loan, Hard Debt', 'Other3',
       'Permanent Second Loan', 'Other5', 'DDF & Other',
       'Historic Tax Credit Equity', 'OHFA HDL', 'Acquired Reserves',
       'Construction Period Income', 'Seller Note', 'Stark County HOME',
       'Columbus City HOME', 'Affordable Housing Trust',
       'City of Columbus/Franklin County', 'FHLB AHP', 'Donations',
       'City of Columbus', 'Franklin County', 'FHLB Cin. AHP',
       'Donations from Huntington and CHN', 'DDF and Other',
       'City of Cleveland Funds', 'Cuyahoga County Funds'], dtype=object)

In [9]:
df = df[df["Source"]!='GP Capital Contribution']
df = df[df["Source"]!='GP Equity'] #drop GP contributions 

In [25]:
#create subcategories for types of financing, below code applies these and then creates a new column 
tax_credit = ['Tax Credit Equity']
historic_tax_credit = ['Historic Tax Credit Equity']
bank_loan = ['Construction Loan','Permanent First Loan, Hard Debt','Permanent Second Loan','Seller Note'] #for affordable housing trust http://hztrust.org/
affordable_housing_trust = ['Affordable Housing Trust']
deferred_dev = ['Deferred Developer Fee','DDF & Other','DDF and Other']
state_credit = ['HDAP', 'HDAP: OHTF/HOME']  #without specifying which type of HDAP its hard to know if its a grant or loan, but seems liek more grant funding than loans https://ohiohome.org/ppd/documents/HDAP-Matrix-2020.pdf
federal_credit = []
federal_home_loan_bank = ['FHLB AHP','FHLB Cin. AHP']
local_grant = ['City of Cleveland Funds', 'Cuyahoga County Funds''City of Columbus/Franklin County','Donations',
       'City of Columbus', 'Franklin County','Stark County HOME', 'Columbus City HOME','Stark County HOME','Donations from Huntington and CHN','Donations'] #Home are grants https://www.bizjournals.com/columbus/news/2020/09/01/affordable-housing-programs-tax-credits-and-home.html, there was only 1 donation and I put it here but I'm not sure  
public_private_partnership = [] 
fed_grant = []

#unmapped Others, 'Construction Period Income' (shows up twice), 'Acquired Reserves' (also shows up twice)

In [19]:
df.head()

,Unnamed: 0,Source,Amount of Funds,Funding Type,File Name,Financing_sub_cat
0,24,Tax Credit Equity,"187,828.00",Construction Financing Sources,PrairieViewApartments,Tax Credit
1,25,Construction Loan,"4,709,536.00",Construction Financing Sources,PrairieViewApartments,Bank Loan
2,26,Other1,"1,250,000.00",Construction Financing Sources,PrairieViewApartments,Other
3,27,Other2,"249,750.00",Construction Financing Sources,PrairieViewApartments,Other
4,28,Other4,"1,275,733.00",Construction Financing Sources,PrairieViewApartments,Other


In [26]:
conditions = [
    df["Source"].isin(tax_credit),
    df["Source"].isin(historic_tax_credit),
    df["Source"].isin(bank_loan),
    df["Source"].isin(affordable_housing_trust),
    df["Source"].isin(deferred_dev),
    df["Source"].isin(state_credit),
    df["Source"].isin(federal_credit),
    df["Source"].isin(federal_home_loan_bank),
    df["Source"].isin(local_grant),
    df["Source"].isin(public_private_partnership),
    df["Source"].isin(fed_grant)]

In [27]:
outputs = ["Tax Credit","Historic Tax Credit",'Bank Loan',"Affordable Housing Trust","Deferred Developer Fee","State Housing Credit Equity",
    "Federal Housing Credit Equity","Federal Home Loan Bank","Local Grant","Public Private Partnership","Federal Grant"]

In [28]:
import numpy as np
df["Financing_sub_cat"] = np.select(conditions, outputs, 'Other')

In [29]:
df.head()

,Unnamed: 0,Source,Amount of Funds,Funding Type,File Name,Financing_sub_cat
0,24,Tax Credit Equity,"187,828.00",Construction Financing Sources,PrairieViewApartments,Tax Credit
1,25,Construction Loan,"4,709,536.00",Construction Financing Sources,PrairieViewApartments,Bank Loan
2,26,Other1,"1,250,000.00",Construction Financing Sources,PrairieViewApartments,Other
3,27,Other2,"249,750.00",Construction Financing Sources,PrairieViewApartments,Other
4,28,Other4,"1,275,733.00",Construction Financing Sources,PrairieViewApartments,Other


In [33]:
df.drop(["Unnamed: 0"],axis=1).to_csv("Ohio_scraped_financing_info_Awarded_Only.csv")